# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [27]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [2]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [3]:
# Import the necessary CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path('Data/neighborhoods_coordinates.csv')
neighborhood_loc_df = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [4]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    avg_housing_units_per_year = sfo_data['housing_units'].groupby(sfo_data.index).mean()
    
    min_value = avg_housing_units_per_year.min()
    max_value = avg_housing_units_per_year.max()
    std_value = round(avg_housing_units_per_year.std())

    avg_housing_units_per_year_plot = avg_housing_units_per_year.hvplot.bar(
        x='year',
        y='housing_units',
        xlabel='Year',
        ylabel='Housing Units',
        ylim=(min_value-std_value, max_value+std_value), 
        title='Housing Units in San Francisco from 2010 to 2016', 
        width = 600
    ).opts(yformatter="%.0f")

    return avg_housing_units_per_year_plot

In [5]:
def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    average_gross_rent_df = sfo_data[['gross_rent']].groupby(sfo_data.index).mean()
    
    
    avg_rent_plot = average_gross_rent_df.hvplot.line(
        x='year', 
        y='gross_rent', 
        xlabel='Year', 
        ylabel='Gross Rent', 
        color='red',
        width = 600,
        title='Average Gross Rent by Year'
    )

    return avg_rent_plot

In [6]:
def average_sales_price():
    """Average Sales Price Per Year."""
    average_sales_price_df = sfo_data[['sale_price_sqr_foot']].groupby(sfo_data.index).mean()
    
    avg_sale_plot = average_sales_price_df.hvplot.line(
        x='year', 
        y='sale_price_sqr_foot', 
        xlabel='Year', 
        ylabel='Price per SqFt', 
        color='purple', 
        width = 600,
        title='Average Price per SqFt by Year'
    )
    
    return avg_sale_plot

In [7]:
def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    sfo_data_new = sfo_data.groupby(
    [sfo_data.index, 'neighborhood']).mean()
    sfo_sale_rent = sfo_data_new[['sale_price_sqr_foot', 'housing_units', 'gross_rent']]    
    df_mean_values = pd.DataFrame(sfo_sale_rent).reset_index() 
                            
    avg_prices_by_neighborhood = df_mean_values.hvplot.line(
        x='year', 
        y='sale_price_sqr_foot', 
        xlabel='Year', 
        ylabel='Avg. Sale Price Per Square Foot', 
        groupby='neighborhood', 
        width=500
    )
    
    return avg_prices_by_neighborhood

In [8]:
def average_gross_rent_by_neighborhood():
    """Average Prices by Neighborhood."""
    sfo_data_new = sfo_data.groupby([sfo_data.index, 'neighborhood']).mean()
    
    sfo_sale_rent = sfo_data_new[['sale_price_sqr_foot', 'housing_units', 'gross_rent']]   
    
    df_mean_values = pd.DataFrame(sfo_sale_rent).reset_index() 
                            
    avg_gross_rent_by_neighborhood = df_mean_values.hvplot.line(
        x='year', 
        y='gross_rent', 
        xlabel='Year', 
        ylabel='Avg. Gross Rent per Year', 
        groupby='neighborhood', 
        width=500
    )
    
    return avg_gross_rent_by_neighborhood

In [9]:
def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    
    df_expensive_neighborhoods = sfo_data[
        ['sale_price_sqr_foot', 'housing_units', 'gross_rent']
    ].groupby(
        sfo_data['neighborhood']
    ).mean()
   
    df_expensive_neighborhoods.sort_values(
        by='sale_price_sqr_foot',
        ascending=False, 
        inplace=True
    )
    
    df_expensive_neighborhoods.reset_index(
        inplace=True
    )
    top_10_expensive_neighborhoods = df_expensive_neighborhoods.head(10)
    
    top_10_max = df_expensive_neighborhoods['sale_price_sqr_foot'].max()
    top_10_std = df_expensive_neighborhoods['sale_price_sqr_foot'].std()

    top_10_neighborhood_plot = top_10_expensive_neighborhoods.hvplot.bar(
        x='neighborhood', 
        y='sale_price_sqr_foot',
        xlabel='Neighborhood', 
        ylabel='Avg Sale Price per Square Foot', 
        title='Top 10 Expensive Neighborhoods in SFO',
        ylim=[0, top_10_max+top_10_std], 
        width=600, rot=90
    )

    return top_10_neighborhood_plot

In [10]:
def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    
    sfo_data_new = sfo_data.groupby(
        [sfo_data.index, 'neighborhood']).mean()
    
    sfo_sale_rent = sfo_data_new[
        ['sale_price_sqr_foot', 'housing_units', 'gross_rent']
    ]    
    
    df_mean_values = pd.DataFrame(sfo_sale_rent).reset_index() 

    most_expensive_neighborhoods_rent_sales_plot = df_mean_values.hvplot.bar(
        x='year', 
        y=['gross_rent', 'sale_price_sqr_foot'],
        xlabel='Neighborhood', 
        ylabel='Num Housing Units',
        title='Top 10 Expensive Neighborhoods in SFO', 
        groupby='neighborhood', 
        width=600, 
        rot=90
    )
    
    return most_expensive_neighborhoods_rent_sales_plot

In [11]:
def parallel_coordinates():
    """Parallel Coordinates Plot."""
    df_expensive_neighborhoods = sfo_data[
        ['sale_price_sqr_foot', 'housing_units', 'gross_rent']
    ].groupby(
        sfo_data['neighborhood']
    ).mean()
   
    df_expensive_neighborhoods.sort_values(
        by='sale_price_sqr_foot',
        ascending=False, 
        inplace=True
    )
    
    df_expensive_neighborhoods.reset_index(
        inplace=True
    )
    top_10_expensive_neighborhoods = df_expensive_neighborhoods.head(10)

    parallel_coordinates = px.parallel_coordinates(
        top_10_expensive_neighborhoods, 
        color='sale_price_sqr_foot', 
        title='Parallel Coordinates Analysis of Most Expensive San Francisco Neighborhoods'
    )

    return pn.pane.Plotly(parallel_coordinates)

In [12]:
def parallel_categories():
    """Parallel Categories Plot."""
    df_expensive_neighborhoods = sfo_data[
        ['sale_price_sqr_foot', 'housing_units', 'gross_rent']
    ].groupby(
        sfo_data['neighborhood']
    ).mean()
   
    df_expensive_neighborhoods.sort_values(
        by='sale_price_sqr_foot',
        ascending=False, 
        inplace=True
    )
    
    df_expensive_neighborhoods.reset_index(
        inplace=True
    )
    top_10_expensive_neighborhoods = df_expensive_neighborhoods.head(10)

    parallel_categories = px.parallel_categories(
        top_10_expensive_neighborhoods,
        dimensions=["neighborhood", "sale_price_sqr_foot", "housing_units", "gross_rent"],
        color="sale_price_sqr_foot",
        color_continuous_scale=px.colors.sequential.Inferno,
        title='Parallel Categories Analysis of Most Expensive San Francisco Neighborhoods',
        labels={
            "neighborhood" : "Neighborhood",
            "sale_price_sqr_foot" : "Sale Price Sqr Foot",
            "housing_units" : "Housing Units",
            "gross_rent" : "Gross Rent"
        },
    )

    return pn.pane.Plotly(parallel_categories)

In [13]:
def neighborhood_map():
    """Neighborhood Map."""
    df_expensive_neighborhoods = sfo_data[['sale_price_sqr_foot', 'housing_units', 'gross_rent']].groupby(
    sfo_data['neighborhood']).mean()
    df_expensive_neighborhoods.sort_values(by='sale_price_sqr_foot', ascending=False, inplace=True)
    df_expensive_neighborhoods.reset_index(inplace=True)
    
    mean_values_neighborhood = df_expensive_neighborhoods[
        ['sale_price_sqr_foot', 'housing_units', 'gross_rent']
    ].groupby(df_expensive_neighborhoods['neighborhood']
    ).mean()
    
    mean_values_neighborhood.reset_index(inplace=True)
    mean_values_neighborhood.head()
    
    combined_df = pd.concat(
        [neighborhood_loc_df, mean_values_neighborhood], 
        axis='columns', 
        join='inner'
    )
    
    combined_df.drop(columns=['neighborhood']).head()
    
    neighborhood_map = px.scatter_mapbox(
        combined_df,
        lat="Lat",
        lon="Lon",
        color="gross_rent",
        size="gross_rent",
        color_continuous_scale=px.colors.cyclical.IceFire,
        size_max=12,
        zoom=11
    )

    return pn.pane.Plotly(neighborhood_map)

In [14]:
def sunburst():
    """Sunburst Plot."""
    
    df_expensive_neighborhoods = sfo_data[
        ['sale_price_sqr_foot', 'housing_units', 'gross_rent']
    ].groupby(sfo_data['neighborhood']
    ).mean()
    
    df_expensive_neighborhoods.sort_values(by='sale_price_sqr_foot', ascending=False, inplace=True)
    df_expensive_neighborhoods.reset_index(inplace=True)
    top_10_expensive_neighborhoods = df_expensive_neighborhoods.head()
    
    df_costs = sfo_data.reset_index()

    df_expensive_neighborhoods_per_year = df_costs[
        df_costs["neighborhood"].isin(top_10_expensive_neighborhoods["neighborhood"])
    ]
    
    sunburst_plot = px.sunburst(
        df_expensive_neighborhoods_per_year,
        path=['year', 'neighborhood'],
        values='sale_price_sqr_foot',
        color='gross_rent',
        color_continuous_scale='RdBu',
        title='Costs Analysis of Most Expensive neighborhoods in San Francisco per Year',
        width=700,
        height=700
    )
    
    return pn.pane.Plotly(sunburst_plot)

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [15]:
# Create a Title for the Dashboard
dashboard_title = '# Real Estate Analysis of San Francisco From 2010 to 2016'

dashboard_text = '''
This dashboard presents a visual analysis of historical prices of house units, 
sale price per square foot and gross rent in San Francisco, California from 2010 to 2016. 
We can navigate through the tabs above to explore more details about the evolution of the 
real estate market on The Golden City across these years.
'''

welcome_row = pn.Column( 
    neighborhood_map()
)

market_analysis_column = pn.Column(
    housing_units_per_year(),  
    average_sales_price(),
    average_gross_rent(),
)

neighborhood_analysis_column = pn.Column(
    average_price_by_neighborhood(),
    average_gross_rent_by_neighborhood(),
    top_most_expensive_neighborhoods(),
    most_expensive_neighborhoods_rent_sales
)

parallel_plot_analysis_column = pn.Column(
    parallel_coordinates(), 
    parallel_categories()
)

sunburst_plot_column = pn.Column(sunburst())

# Create a tab layout for the dashboard
tab = pn.Tabs(
        ("Welcome", welcome_row),
        ("Yearly Market Analysis", market_analysis_column),
        ("Neighborhood Analysis", neighborhood_analysis_column),
        ("Parallel Plot Analysis", parallel_plot_analysis_column),
        ("Sunburst Plot Analysis", sunburst_plot_column)
)

# Create the dashboard
dashboard = pn.Column(
    dashboard_title, 
    dashboard_text,
    tab
)

## Serve the Panel Dashboard

In [16]:
# Serve the dashboard
dashboard.servable()

Column
    [0] Markdown(str)
    [1] Markdown(str)
    [2] Tabs
        [0] Column
            [0] Plotly(Figure)
        [1] Column
            [0] HoloViews(Bars)
            [1] HoloViews(Curve)
            [2] HoloViews(Curve)
        [2] Column
            [0] Row
                [0] HoloViews(DynamicMap)
                [1] Column
                    [0] WidgetBox
                        [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                    [1] VSpacer()
            [1] Row
                [0] HoloViews(DynamicMap)
                [1] Column
                    [0] WidgetBox
                        [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                    [1] VSpacer()
            [2] HoloViews(Bars)
            [3] Column
                [0] Column()
                [1] Row
                    [0] Row(name='interactive01981')
                        [0] HoloViews(DynamicMap, name='interactive01981')
                        [1] Column
                            [0] WidgetBox
                                [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                            [1] VSpacer()
        [3] Column
            [0] Plotly(Figure)
            [1] Plotly(Figure)
        [4] Column
            [0] Plotly(Figure)

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [17]:
housing_units_per_year()

:Bars   [year]   (housing_units)

In [18]:
average_gross_rent()

:Curve   [year]   (gross_rent)

In [19]:
average_sales_price()

:Curve   [year]   (sale_price_sqr_foot)

In [20]:
average_price_by_neighborhood()

:DynamicMap   [neighborhood]
   :Curve   [year]   (sale_price_sqr_foot)

In [21]:
top_most_expensive_neighborhoods()

:Bars   [neighborhood]   (sale_price_sqr_foot)

In [22]:
most_expensive_neighborhoods_rent_sales()

:DynamicMap   [neighborhood]
   :Bars   [year,Variable]   (value)

In [23]:
neighborhood_map()

Plotly(Figure)

In [24]:
parallel_categories()

Plotly(Figure)

In [25]:
parallel_coordinates()

Plotly(Figure)

In [26]:
sunburst()

Plotly(Figure)